In [1]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
import numpy as np
from sklearn.preprocessing import label_binarize
from itertools import cycle
import matplotlib.pyplot as plt
from sklearn.metrics import PrecisionRecallDisplay

In [ ]:
order = pd.read_csv('order.txt', sep='\t')
order = order[['ExternalId', 'IndicationForStudy']]
order['IndicationForStudy'] = order['IndicationForStudy'].str.strip()
order.head()

In [3]:
panel = pd.read_csv('panel.tab', sep='\t')
panel = panel[['ExternalId', 'PanelType']]
panel.head()

,ExternalId,PanelType
0,CPDC151546,HEME
1,CPDC151547,HEME
2,CPDC151548,HEME
3,CPDC151550,SOLID
4,CPDC151555,SOLID


In [4]:
adx = pd.merge(order, panel, on='ExternalId', how='inner')
adx = adx[adx['PanelType'] == 'SOLID']

In [5]:
adx.dropna(subset=['IndicationForStudy'], inplace=True)
breast = adx[adx['IndicationForStudy'].str.contains('Breast', case=False)] #will capture all breast events regardless of EHR annotation
breast.to_csv('adx_breast.txt', sep='\t', index=False)

In [7]:
test50 = pd.read_csv('../data/test50.csv', sep=',')

In [8]:
bcpdids = breast['ExternalId'].tolist()
t50_breast = test50[test50['CPDID'].isin(bcpdids)]


In [33]:
#get the coutns of each diagnosis for solid panels
vc = adx['IndicationForStudy'].value_counts()
adx['DiagnosisCounts'] = adx['IndicationForStudy'].map(vc)
sdx = adx[adx['DiagnosisCounts'] > 10]

In [34]:
import pandas as pd

# Step 1: Use pd.qcut to assign quartile categories
cdx = sdx.copy()
cdx['Quartile'] = pd.qcut(sdx['DiagnosisCounts'], 4, labels=False)

# Step 2: Filter the DataFrame into four separate DataFrames based on the quartile categories
df_q1 = cdx[cdx['Quartile'] == 0]
df_q2 = cdx[cdx['Quartile'] == 1]
df_q3 = cdx[cdx['Quartile'] == 2]
df_q4 = cdx[cdx['Quartile'] == 3]

In [35]:
dxc = cdx[['IndicationForStudy', 'DiagnosisCounts']]
dxc.drop_duplicates(inplace=True)
dxc.to_csv('SolidTumorTrainingDiagnosisCounts.tsv', sep='\t', index=False)

C:\Users\ash\AppData\Local\Temp\ipykernel_31456\2710995269.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dxc.drop_duplicates(inplace=True)


In [36]:
#lets create a dataframe for each quartile
q1_ids = df_q1['ExternalId'].tolist()
q2_ids = df_q2['ExternalId'].tolist()
q3_ids = df_q3['ExternalId'].tolist()
q4_ids = df_q4['ExternalId'].tolist()

In [37]:
compM = CatBoostClassifier()
compM.load_model('../../models/azurify_hg19.0.99.json', format='json')

In [38]:
def get_predictions(data, ids, model):
    data = data.sort_values('CATEGORIZATION') # sort the data ensure class order for plotting
    data = data[data['CPDID'].isin(ids)]
    truth = data['CATEGORIZATION']
    ds = data[['PCHANGE', 'GENE', 'Domain', 'ClinicalSignificance','EFFECT', 'Civic_Evidence', 'FAF', 'GNOMAD_AC', 'GNOMAD_AF', 'EXON_Rank', 'COSMIC_CNT','MVP_score', 'Civic_Drug', 'PMID_COUNT', 'KEGG']]
    
    score = model.predict_proba(ds)

    return score, truth

In [39]:
q1s, q1t = get_predictions(test50, q1_ids, compM)
q2s,q2t = get_predictions(test50, q2_ids, compM)
q3s, q3t = get_predictions(test50, q3_ids, compM)
q4s, q4t = get_predictions(test50, q4_ids, compM)
bs, bt = get_predictions(t50_breast, bcpdids, compM)

In [40]:
def make_pr_curve(y_score, y_test, title):
    n_classes = 5
    precision = dict()
    recall = dict()
    average_precision = dict()
    classes = y_test.unique()
    y_test_bin = label_binarize(y_test, classes=classes)

    plt.rcParams['pdf.fonttype'] = 42
    plt.rcParams['ps.fonttype'] = 42
    plt.rcParams['font.family'] = 'Arial'
    plt.rcParams["figure.figsize"] = (8,6)
    plt.rcParams.update({'font.size': 16})

    for i in range(n_classes):
        precision[i], recall[i], _ = precision_recall_curve(y_test_bin[:, i], y_score[:, i])
        average_precision[i] = average_precision_score(y_test_bin[:, i], y_score[:, i])

    # A "micro-average": quantifying score on all classes jointly
    precision["micro"], recall["micro"], _ = precision_recall_curve(
        y_test_bin.ravel(), y_score.ravel()
    )
    average_precision["micro"] = average_precision_score(y_test_bin, y_score, average="micro")

    # setup plot details
    colors = cycle(["#282A74", "#991B1E", "#6CA9DC", "#FFCE06", "#8A8D8F"])

    _, ax = plt.subplots(figsize=(10, 8))

    f_scores = np.linspace(0.2, 0.8, num=4)
    lines, labels = [], []
    for f_score in f_scores:
        x = np.linspace(0.01, 1)
        y = f_score * x / (2 * x - f_score)
        (l,) = plt.plot(x[y >= 0], y[y >= 0], color="gray", alpha=0.2)
        plt.annotate("f1={0:0.1f}".format(f_score), xy=(0.9, y[45] + 0.02))

    display = PrecisionRecallDisplay(
        recall=recall["micro"],
        precision=precision["micro"],
        average_precision=average_precision["micro"],
    )
    display.plot(ax=ax, name="Micro-average", color="green")

    nl = ['Benign', 'Pathogenic', 'Likely Benign', 'Likely Pathogenic', 'VUS']

    for i, color in zip(range(len(classes)), colors):
        display = PrecisionRecallDisplay(
            recall=recall[i],
            precision=precision[i],
            average_precision=average_precision[i],
        )
        display.plot(ax=ax, name=f"{nl[i]}", color=color)

    # add the legend for the iso-f1 curves
    handles, labels = display.ax_.get_legend_handles_labels()
    handles.extend([l])
    labels.extend(["iso-f1 curves"])
    # set the legend and the axes
    ax.legend(handles=handles, labels=labels, loc="lower left")
    ax.set_title(title)
    plt.savefig(title +'.pdf', dpi=600)
    plt.clf()
    plt.close("all")

In [41]:
make_pr_curve(q1s, q1t, "Q1 Diagnoses Frequency Precision-Recall")
make_pr_curve(q2s, q2t, "Q2 Diagnoses Frequency Precision-Recall")
make_pr_curve(q3s, q3t, "Q3 Diagnoses Frequency Precision-Recall")
make_pr_curve(q4s, q4t, "Q4 Diagnoses Frequency Precision-Recall")
make_pr_curve(bs, bt, "Breast Cancer Precision-Recal")

In [46]:
#total test set PR curve and an individual example
ts = test50.sort_values('CATEGORIZATION') # sort the data ensure class order for plotting
y_test = ts['CATEGORIZATION']
s = ts[['PCHANGE', 'GENE', 'Domain', 'ClinicalSignificance','EFFECT', 'Civic_Evidence', 'FAF', 'GNOMAD_AC', 'GNOMAD_AF', 'EXON_Rank', 'COSMIC_CNT','MVP_score', 'Civic_Drug', 'PMID_COUNT', 'KEGG']]
y_score = compM.predict_proba(s)


In [47]:
# For each class
n_classes = 5
precision = dict()
recall = dict()
average_precision = dict()
y_test_bin = label_binarize(y_test, classes=(y_test.unique()))

for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(y_test_bin[:, i], y_score[:, i])
    average_precision[i] = average_precision_score(y_test_bin[:, i], y_score[:, i])

# A "micro-average": quantifying score on all classes jointly
precision["micro"], recall["micro"], _ = precision_recall_curve(
    y_test_bin.ravel(), y_score.ravel()
)
average_precision["micro"] = average_precision_score(y_test_bin, y_score, average="micro")

In [ ]:
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['font.family'] = 'Arial'
plt.rcParams["figure.figsize"] = (8,6)
plt.rcParams.update({'font.size': 16})

# setup plot details
colors = cycle(["#282A74", "#991B1E", "#6CA9DC", "#FFCE06", "#8A8D8F"])

_, ax = plt.subplots(figsize=(10, 8))

f_scores = np.linspace(0.2, 0.8, num=4)
lines, labels = [], []
for f_score in f_scores:
    x = np.linspace(0.01, 1)
    y = f_score * x / (2 * x - f_score)
    (l,) = plt.plot(x[y >= 0], y[y >= 0], color="gray", alpha=0.2)
    plt.annotate("f1={0:0.1f}".format(f_score), xy=(0.9, y[45] + 0.02))

display = PrecisionRecallDisplay(
    recall=recall["micro"],
    precision=precision["micro"],
    average_precision=average_precision["micro"],
)
display.plot(ax=ax, name="Micro-average", color="green")

nl = ['Benign', 'Pathogenic', 'Likely Benign', 'Likely Pathogenic', 'VUS']


for i, color in zip(range(n_classes), colors):
    display = PrecisionRecallDisplay(
        recall=recall[i],
        precision=precision[i],
        average_precision=average_precision[i],
    )
    display.plot(ax=ax, name=f"{nl[i]}", color=color)

# add the legend for the iso-f1 curves
handles, labels = display.ax_.get_legend_handles_labels()
handles.extend([l])
labels.extend(["iso-f1 curves"])
# set the legend and the axes
ax.legend(handles=handles, labels=labels, loc="lower left")
ax.set_title("Test Data Precision-Recall")
plt.savefig("Test Data Precision Recall.pdf", dpi=600)

plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
y_pred = compM.predict(s)
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
# printing metrics
print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

